## FFT Model demonstration
#### The following is a brief demonstration of the newly created FFT forecasting model. This model is especially suited for data that is very seasonal. The datasets chosen for this demonstration were selected accordingly.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from u8timeseries import TimeSeries
from u8timeseries.models import (
    FFT,
    AutoARIMA,
    ExponentialSmoothing,
    Prophet,
    Theta
)
from u8timeseries.metrics import mape
from u8timeseries.utils.missing_values import auto_fillna

#### Read data and fill gaps (you can safely ignore this part, it was not made to look nice)

In [ ]:
# create series
df = pd.read_csv('temps.csv')
dates = df['Daily minimum temperatures'].str.replace('^[^\d]*', '').astype(float)
dates.index = pd.to_datetime(df['Date'])

# find missing dates
delta = pd.date_range(start = '1981-01-01', end = '1990-12-31' ).difference(dates.index)

# add missing dates
delta_vals = pd.Series([np.mean(dates)] * len(delta))
delta_vals.index = delta
dates = dates.append(delta_vals)
dates = dates.sort_index()

#### Constructing our TimeSeries instance and a train and validation set

In [ ]:
ts = TimeSeries.from_times_and_values(dates.index, dates)
train, val = ts.split_after(pd.Timestamp('19850701'))
train.plot()
val.plot()

### Basic FFT model

In [ ]:
model = FFT(required_matches=set(), nr_freqs_to_keep=None)
model.fit(train)
pred_val = model.predict(len(val))

### Results:
#### The plot below shows us that a simple DFT with a random train-test split will most likely lead to bad results. Upon closer inspection we can see that the prediction (in green) simply repeats the training set (blue). This is the standard behavior of the DFT, and by itself it is quite useless, since repeating our training set could be done much more efficiently. Two improvements were made to this approach.

In [ ]:
train.plot()
val.plot()
pred_val.plot()
print("MAPE:", mape(pred_val, val))

### Improvement 1: Crop the training set
#### The first improvement consists of cropping the training set before feeding it to the FFT algorithm such that the first timestamp in the cropped series matches the first timestamp to be predicted in terms of seasonality, i.e. it has the same month, day, weekday, time of day, etc. We can achieve this by passing the optional argument 'required_matches' to the FFT constructor that explicitly tells our model which timestamp attributes are relevant. If we don't set it manually, the model will attempt to automatically find the pd.Timestamp attributes that are relevant and crop the training set accordingly (which we will do here).

In [ ]:
model = FFT(nr_freqs_to_keep=None)
model.fit(train)
pred_val = model.predict(len(val))

### Results:
#### We can see that the results look like the seasonality of the predictions nicely aligns with the seasonality of the validation set. However, we are still just repeating the training set, including all of the noise. Looking at the error we can see that this is still a very bad forecast.

In [ ]:
train.plot()
val.plot()
pred_val.plot()
print("MAPE:", mape(pred_val, val))

### Improvement 2: Filtering out low-amplitude waves
#### The decomposition of the DFT into the frequency domain allows us to selectively filter out waves with low amplitudes. This allows us to keep strong seasonal trends while discarding some noise. This is achieved in the FFT model by passing the optional argument 'filter_first_n'. This argument represents the total number of frequencies that will be kept. For instance, if a value of 20 is passed, only the 20 frequencies with the highest amplitudes will be utilized. The default value is set to 10.

In [ ]:
model = FFT(nr_freqs_to_keep=20)
model.fit(train)
pred_val = model.predict(len(val))

### Results:
#### We get a signal that is less noisy. Depending on the data set, this might be a better forecast. Looking at the error metric, we can see that this model performs significantly better than the previous models.

In [ ]:
train.plot()
val.plot()
pred_val.plot()
print("MAPE:", mape(pred_val, val))

### New Data: Monthly Air Passengers
#### Let's try out a different data set that has a global upward trend

In [ ]:
df = pd.read_csv('AirPassengers.csv', delimiter=",")
ts_2 = TimeSeries.from_dataframe(df, 'Month', '#Passengers')
train, val = ts_2.split_after(pd.Timestamp('19551201'))
train.plot()
val.plot()

In [ ]:
model = FFT()
model.fit(train)
pred_val = model.predict(len(val))

#### Clearly, our model fails completely at incorporating the upward trend. Due to the trend, our model also fails to recognize the monthly seasonality.

In [ ]:
train.plot()
val.plot()
pred_val.plot()

#### This problem can be solved by setting the optional trend argument to either 'poly' or 'exp', which fits a polynomial or exponential funtion to the data and subtracts it before moving on to DFT. When predicting, the trend is added again.

In [ ]:
model = FFT(trend='poly')
model.fit(train)
pred_val = model.predict(len(val))

### Result:
#### We have a much better prediction now.

In [ ]:
train.plot()
val.plot()
pred_val.plot()

### New Data: Hourly Nuclear Energy Generation

In [ ]:
df = pd.read_csv('energy_dataset.csv', delimiter=",")
df['time'] = pd.to_datetime(df['time'], utc=True)
df['time']= df.time.dt.tz_localize(None)

ts_3 = TimeSeries.from_dataframe(df, 'time', 'generation nuclear')
ts_3 = auto_fillna(ts_3)
train, val = ts_3.split_after(pd.Timestamp('2017-07-01'))
train.plot()
val.plot()

### Forecasting Performance:
#### Instead of simply looking at the performance of the FFT model, l decided to look at how each of the currently implemented forecasting models performs on this new data set in terms of MAPE. Surprisingly, on this dataset, the FFT model outperforms all of the others. Granted, this dataset was specifically chosen because of its highly seasonal nature. However, this shows us that there definitely are use cases for FFT. Furthermore, the FFT model has a much shorter running time than the other models.

In [ ]:
models = [
    AutoARIMA(),
    Prophet(),
    ExponentialSmoothing(),
    Theta(),
    FFT()
]

for model in models:
    model.fit(train)
    pred_val = model.predict(len(val))
    print(str(model) + " MAPE: " + str(mape(pred_val, val)))
